# Image classifier

This notebook will cover basically the same steps as the previous one, but this time we will use a different dataset and a different model. We will use the [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset, which consists of 60,000 32x32 color images in 10 classes, with 6,000 images per class. The dataset is divided into 50,000 training images and 10,000 testing images. The classes are mutually exclusive and there is no overlap between them.

My name is Bruno, aka (Awhux), and this is a course I'm taking to learn more about Machine Learning and Deep Learning. I'm using this notebook to take notes and to share my experience with others. I hope you enjoy it!

## Useful notes

- [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset
- [CIFAR-10 python version](https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz)

- [TensorFlow](https://www.tensorflow.org/)
- [Keras](https://keras.io/)

- [Google Colab](https://colab.research.google.com/)


## Config


In [ ]:
# When you use colab, you can easily change some options here.
# If you're seen this code, just double click in this cell and change the options.

# If you're using vscode, you may need update some options listed here

# If you're using vscode, you may need to set this to True
install_packages = False  # @param {type:"boolean"}
use_gpu = True  # @param {type:"boolean"}

images_download_url = 'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz'
download_file_name = 'cifar-10-python.tar.gz'
extract_to = 'images'

## Imports


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import os
import copy
import tarfile

import requests

import pickle

from zipfile import ZipFile

In [ ]:
if use_gpu:
    gpus = tf.config.experimental.list_physical_devices('GPU')

    if len(gpus) >= 1:
        try:
            # Enable all GPUs
            tf.config.experimental.set_visible_devices(gpus, 'GPU')

            # Set memory growth
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
                print(f"Enabled {gpu.name}")
        except RuntimeError as e:
            # Visible devices must be set before GPUs have been initialized
            print(e)
else:
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Utils functions


In [ ]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


"""
Download and extract our images
"""


def get_download_progress(count, block_size, total_size):
    percent = int(count * block_size * 100 / total_size)
    print(f"\r{bcolors.OKCYAN}Downloading images: {percent}%{bcolors.ENDC}", end='')


def download_images():
    if not os.path.exists(extract_to):
        os.mkdir(extract_to)
    if not os.path.exists(os.path.join(extract_to, download_file_name)):
        r = requests.get(images_download_url, stream=True)
        with open(os.path.join(extract_to, download_file_name), 'wb') as f:
            total_length = int(r.headers.get('content-length'))
            for chunk in r.iter_content(chunk_size=1024):
                f.write(chunk)
                get_download_progress(f.tell(), 1024, total_length)
        print(f"\r{bcolors.OKGREEN}Downloaded images{bcolors.ENDC}")
    else:
        print(f"{bcolors.OKGREEN}Images already downloaded{bcolors.ENDC}")

    if not os.path.exists(os.path.join(extract_to, 'cifar-10-batches-py')):
        print("Extracting images...")
        tar = tarfile.open(os.path.join(
            extract_to, download_file_name), "r:gz")
        tar.extractall(extract_to)
        tar.close()
    else:
        print(f"{bcolors.OKGREEN}Images already extracted{bcolors.ENDC}")


def unpicke_images():
    images = []
    labels = []
    for i in range(1, 6):
        with open(os.path.join(extract_to, 'cifar-10-batches-py', f'data_batch_{i}'), 'rb') as f:
            data = pickle.load(f, encoding='bytes')
            images.append(data[b'data'])
            labels.append(data[b'labels'])
    images = np.concatenate(images)
    labels = np.concatenate(labels)
    return images, labels


class print_colored:
    def __init__(self, color):
        self.color = color

    def __enter__(self):
        print(self.color, end='')

    def __exit__(self, exc_type, exc_val, exc_tb):
        print(bcolors.ENDC, end='')


download_images()

# print(f"{bcolors.OKGREEN}Images shape: {images.shape}{bcolors.ENDC}")
# Images shape: (50000, 3072)
# print(f"{bcolors.OKGREEN}Labels shape: {labels.shape}{bcolors.ENDC}")
# Labels shape: (50000,)

## Loading image dataset


In [ ]:
def load_images():
    import random

    # Unpickle images, basically this is a numpy array
    # with shape (50000, 3072), we want to reshape it to
    # (50000, 3, 32, 32) and move the axis to the last.
    # Why we need to do this? Because we want to use
    # Conv2D layer in our model, and Conv2D layer
    # requires the input shape to be (batch_size, height, width, channels)
    # So we need to reshape our images to (50000, 32, 32, 3),
    # 50000 is the batch size (total images), 32 is the height, 32 is the width,
    # and 3 is the channels (RGB)

    # But why won't we change the labels shape too?
    # Because we don't need to, we're not going to use Conv2D layer
    # for our labels, we're going to use Dense layer, and Dense layer
    # requires the input shape to be (batch_size, input_size)
    # So we don't need to reshape our labels, we just need to
    # make sure that the input_size is 3072, and it is.
    images, labels = unpicke_images()
    images = images.reshape((images.shape[0], 3, 32, 32))
    images = np.moveaxis(images, 1, -1)

    # Before we return our images, we need to randomize it (shuffle)
    # Why? Because we want to split our images into train and test set
    # and we don't want the train set to be all cats and the test set to be all dogs
    # So we need to randomize it first
    # We're going to use np.random.permutation to randomize our images
    # and we're going to use np.random.seed to make sure that the randomization
    # is the same every time we run this code

    random_seed = random.randint(1, 50000)

    np.random.seed(random_seed)
    randomize = np.random.permutation(len(images))
    images = images[randomize]
    labels = labels[randomize]

    return images, labels


images, labels = load_images()

with print_colored(bcolors.OKGREEN):
    print(f"Images shape: {images.shape}")
    print(f"Labels shape: {labels.shape}")

In [ ]:
# Lets show the first 5 images with their labels
for i in range(5):
    print(f"{bcolors.OKGREEN}Image label: {labels[i]}{bcolors.ENDC}")
    plt.imshow(images[i])
    plt.show()

In [ ]:
# Normalize our data
images = images / 255.0

# Let's check if our data is normalized
with print_colored(bcolors.OKGREEN):
    print(f"Max value: {np.max(images)}")
    print(f"Min value: {np.min(images)}")

## Preprocessing


In [ ]:
# Lets split our data into train and test sets
# We're going to use 80% of our data for training
# and 20% of our data for testing
train_images, test_images, validation_images = np.split(images, [int(
    len(images) * 0.8), int(len(images) * 0.9)])
train_labels, test_labels, validation_labels = np.split(labels, [int(
    len(labels) * 0.8), int(len(labels) * 0.9)])

with print_colored(bcolors.OKGREEN):
    print(f"Train images shape: {train_images.shape}")
    print(f"Train labels shape: {train_labels.shape}")
    print(f"Test images shape: {test_images.shape}")
    print(f"Test labels shape: {test_labels.shape}")
    print(f"Validation images shape: {validation_images.shape}")
    print(f"Validation labels shape: {validation_labels.shape}")

    # We should have 40000 images for training, 10000 images for testing,

## Building the model


In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.6),
    Dense(10, activation='softmax')
])

# Our model:
# 1. Conv2D layer with 32 filters, kernel size 3x3, relu activation function, and input shape (32, 32, 3)
# 1.1 Those 32 filters will be applied to our input image, and each filter will produce a 2D activation map
# 1.2 The kernel size 3x3 means that the filter will be 3x3 pixels
# 1.3 The relu activation function will be applied to the activation map, and it will remove all negative values
# 1.4 The input shape is (32, 32, 3), 32 is the height, 32 is the width, and 3 is the channels (RGB)
# 2. MaxPooling2D layer with pool size 2x2
# 2.1 This layer will reduce the size of the activation map by 2x2
# 3. Conv2D layer with 64 filters, kernel size 3x3, relu activation function
# 4. MaxPooling2D layer with pool size 2x2
# 5. Conv2D layer with 64 filters, kernel size 3x3, relu activation function
# 6. Flatten layer
# 6.1 This layer will flatten our 2D activation map to 1D
# 7. Dense layer with 64 neurons, relu activation function
# 7.1 This layer will connect all neurons from the previous layer to this layer
# 8. Dropout layer with 0.5 rate
# 8.1 This layer will randomly drop 50% of the neurons
# 9. Dense layer with 10 neurons, softmax activation function
# 9.1 This layer will connect all neurons from the previous layer to this layer, softmax activation function will
# normalize the output values to 0-1, and the sum of all output values will be 1

In [ ]:
# Now let's compile our model
model.compile(tf.keras.optimizers.Adam(0.001),
              loss=tf.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

# We're using Adam optimizer with learning rate 0.001, and sparse_categorical_crossentropy loss function
# We're using sparse_categorical_crossentropy because our labels are not one-hot encoded

In [ ]:
# Let's get a summary of our model
model.summary()

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir='logs', histogram_freq=1)

In [ ]:
# Time to train our model
history = model.fit(train_images, train_labels, epochs=50, validation_data=(
    validation_images, validation_labels), callbacks=[tensorboard_callback], verbose=2)
# history = model.fit(train_images, train_labels, epochs=50, validation_split=0.2, callbacks=[tensorboard_callback], verbose=2)

In [ ]:
def plot_history(history):
    fig, axs = plt.subplots(2)

    # create accuracy subplot
    axs[0].plot(history.history["accuracy"], label="train accuracy")
    axs[0].plot(history.history["val_accuracy"], label="test accuracy")
    axs[0].set_ylabel("Accuracy")
    axs[0].legend(loc="lower right")
    axs[0].set_title("Accuracy eval")

    # create error subplot
    axs[1].plot(history.history["loss"], label="train error")
    axs[1].plot(history.history["val_loss"], label="test error")
    axs[1].set_ylabel("Error")
    axs[1].set_xlabel("Epoch")
    axs[1].legend(loc="upper right")
    axs[1].set_title("Error eval")

    plt.show()

In [ ]:
plot_history(history)

with print_colored(bcolors.OKGREEN):
    print(f"Test loss: {history.history['val_loss'][-1]}")
    print(f"Test accuracy: {history.history['val_accuracy'][-1]}")

    print(f"Train loss: {history.history['loss'][-1]}")
    print(f"Train accuracy: {history.history['accuracy'][-1]}")

## Evaluating the model


In [ ]:
from tensorflow.keras.metrics import Precision, Recall

precision = Precision()
recall = Recall()

for test in test_images:
    X, y = test.reshape(1, 32, 32, 3), test_labels[0]
    y_pred = model.predict(X)
    y_pred = np.argmax(y_pred, axis=1)[0]
    precision.update_state([y], [y_pred])
    recall.update_state([y], [y_pred])

with print_colored(bcolors.OKGREEN):
    print(f"Precision: {precision.result().numpy()}")
    print(f"Recall: {recall.result().numpy()}")

In [ ]:
# Time to test it ourselves!
# We'll download some images on the internet, resize and test it

# Download some images
from PIL import Image


def download_test_images():
    car_image_url = 'https://cdni.autocarindia.com/utils/imageresizer.ashx?n=https://cms.haymarketindia.net/model/uploads/modelimages/Hyundai-Grand-i10-Nios-200120231541.jpg&w=32&h=32&q=75&c=1'
    dog_image_url = 'https://cdn.pixabay.com/photo/2016/12/13/05/15/puppy-1903313_960_720.jpg'
    cat_image_url = 'https://cdn.pixabay.com/photo/2017/02/20/18/03/cat-2083492_960_720.jpg'

    car_image_file = 'car.jpg'
    dog_image_file = 'dog.jpg'
    cat_image_file = 'cat.jpg'

    if not os.path.exists(car_image_file):
        r = requests.get(car_image_url, stream=True)
        with open(car_image_file, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024):
                f.write(chunk)

    if not os.path.exists(dog_image_file):
        r = requests.get(dog_image_url, stream=True)
        with open(dog_image_file, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024):
                f.write(chunk)

    if not os.path.exists(cat_image_file):
        r = requests.get(cat_image_url, stream=True)
        with open(cat_image_file, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024):
                f.write(chunk)

    return car_image_file, dog_image_file, cat_image_file


car_image_file, dog_image_file, cat_image_file = download_test_images()

# Resize the images


def resize_image(image_file):
    image = Image.open(image_file)
    image = image.resize((32, 32))
    image.save(image_file)


resize_image(car_image_file)
resize_image(dog_image_file)
resize_image(cat_image_file)

# Load the images


def load_test_images():
    images = []
    for image_file in [car_image_file, dog_image_file, cat_image_file]:
        image = Image.open(image_file)
        image = np.array(image)
        images.append(image)
    return np.array(images)


test_images = load_test_images()

# Show the images
for image in test_images:
    plt.imshow(image)
    plt.show()

# Normalize the images
test_images = test_images / 255.0

# Predict the images
predictions = model.predict(test_images)

# Show the predictions
for i, prediction in enumerate(predictions):
    print(f"{bcolors.OKGREEN}Prediction: {np.argmax(prediction)}{bcolors.ENDC}")
    plt.imshow(test_images[i])
    plt.show()

## Saving the model


In [ ]:
# Now our job is done, we can save our model
model.save('model.h5')

# If you want to recover the last training session, you can use this code
# model = tf.keras.models.load_model('model.h5')